# Simulation experiment using noise-corrected data

Run entire simulation experiment multiple times to generate confidence interval.  The simulation experiment can be found in ```functions/pipeline.py```

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Parameters
dataset_name = "Human_analysis"
analysis_name = 'Human_sample_lvl_sim'
NN_architecture = 'NN_2500_30'
file_prefix = 'Experiment_corrected'
num_simulated_samples = 500
lst_num_experiments = [1, 2, 5, 10, 20,
                     50, 100, 250, 500]

corrected = True
use_pca = True
num_PCs = 10
local_dir = os.path.abspath(
      os.path.join(
          os.getcwd(), "../../../../"))

iterations = range(5) 
num_cores = 5

In [3]:
# Input
base_dir = os.path.abspath(
  os.path.join(
      os.getcwd(), "../.."))    # base dir on repo


normalized_data_file = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "input",
    "recount2_gene_normalized_data.tsv.xz")

In [4]:
# Output files
similarity_corrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "Human_sample_lvl_sim_similarity_corrected.pickle")

ci_corrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "Human_sample_lvl_sim_ci_corrected.pickle")

In [5]:
# Run multiple simulations - corrected
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.sample_level_simulation_corrected)(i,
                                                     NN_architecture,
                                                     dataset_name,
                                                     analysis_name,
                                                     num_simulated_samples,
                                                     lst_num_experiments,
                                                     corrected,
                                                     use_pca,
                                                     num_PCs,
                                                     file_prefix,
                                                     normalized_data_file,
                                                     local_dir) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 84.7min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 84.8min remaining: 127.2min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 84.9min remaining: 56.6min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 85.0min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 85.0min finished


In [6]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of experiments,,,,,
1,0.999993,0.999992,0.999993,0.999993,0.999993
2,0.999303,0.999024,0.998870,0.998474,0.999394
5,0.996534,0.997102,0.995776,0.995082,0.994886
10,0.989693,0.987923,0.988196,0.991593,0.991856
20,0.982375,0.984186,0.979648,0.984506,0.980886
50,0.951039,0.947344,0.947615,0.945428,0.951915
100,0.889803,0.895689,0.891160,0.901241,0.890995
250,0.706142,0.708936,0.702330,0.700476,0.691935
500,0.121798,0.115176,0.111010,0.117815,0.132291


In [7]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of experiments,
1,0.999993
2,0.999013
5,0.995876
10,0.989852
20,0.982320
50,0.948668
100,0.893778
250,0.701964
500,0.119618


In [8]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of experiments,
1,8.688055e-08
2,1.162764e-04
5,2.982402e-04
10,5.816734e-04
20,6.606559e-04
50,8.590902e-04
100,1.497916e-03
250,2.055431e-03
500,2.561482e-03


In [9]:
# Get confidence interval for each row (number of experiments)
# z-score for 95% confidence interval
err = std_scores*1.96

In [10]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of experiments,,
1,0.999993,0.999993
2,0.998785,0.999241
5,0.995291,0.996461
10,0.988712,0.990992
20,0.981025,0.983615
50,0.946984,0.950352
100,0.890842,0.896713
250,0.697935,0.705992
500,0.114598,0.124639


In [11]:
mean_scores

,score
number of experiments,
1,0.999993
2,0.999013
5,0.995876
10,0.989852
20,0.982320
50,0.948668
100,0.893778
250,0.701964
500,0.119618


In [12]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_corrected_file)
ci.to_pickle(ci_corrected_file)